<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/tidy_up_preprocessing_notebook/notebooks/processed/ct_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [358]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-02-13
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Importing necessary libraries and downloading NLTK data.
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-02-13\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Importing necessary libraries and downloading NLTK data.\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [359]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Function**

preprocessor function : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [360]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


find_row: This function searches upwards from the given current_row_num in the DataFrame df to find the first row where the value in column "M" is "A". It returns the index of that row. If no such row is found, it returns 0

In [361]:
def find_row (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "A":
      break
    else:
      i-=1
  return i

find_row_empty: This function searches upwards from the given current_row_num to find the first row where col1 has the value "A" and col2 is not an empty list. It returns the index of that row.

In [362]:
def find_row_empty (df, col1, col2, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col1][i] == "A" and df[col2][i] != []:
      break
    else:
      i-=1
  return i

In [363]:
def create_ques_num_column (data, new_col,marker_col):
  #Create question number column
  data[new_col]=None
  #set global var to count question number
  num = 0
  #if Q was found, num adds 1 otherwise none
  for i in data.index:
    if data.loc[i,marker_col]=="Q":
      data.at[i,new_col]=num
      num +=1
    else:
      continue

## **Data**

In [364]:
#drive.mount('/content/drive')

In [365]:
#!ls"/content/bank_of_england/data/preprocessed_data/Archived/jpmorgan_qa_section_preprocessed.csv"

JP Morgan QA section

In [366]:
#Obtaining management discussion / git bash
!git clone https://github.com/sheldonkemper/bank_of_england.git
!git switch Preprocessing
%cd bank_of_england/data/preprocessed_data/archived
%ls

Cloning into 'bank_of_england'...
remote: Enumerating objects: 1173, done.
remote: Counting objects: 100% (331/331), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 1173 (delta 212), reused 88 (delta 69), pack-reused 842 (from 2)
Receiving objects: 100% (1173/1173), 10.18 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (576/576), done.
fatal: invalid reference: Preprocessing
/content/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_england/data/preprocessed_data/archived/bank_of_englan

In [367]:
#Defining qa_data
qa_data = pd.read_csv("jpmorgan_qa_section_preprocessed.csv")
qa_data.head()

,speaker,marker,job_title,utterance,filename,financial_quarter,call_date,tokenised_data,cleaned_data
0,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","Yeah. I think the conventional wisdom on QT, a...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['yeah', 'think', 'conventional', 'wisdom', 'p...",['yeah think conventional wisdom qt im pretend...
1,Mike Mayo,Q,"Analyst, Wells Fargo Securities LLC","So, you'll stay around maybe for a few more ye...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['stay', 'around', 'maybe', 'years', 'base', '...",['so stay around maybe years base case right n...
2,Mike Mayo,Q,"Analyst, Wells Fargo Securities LLC",All right. Thank you.,4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['right', 'thank', 'you']",['right thank you']
3,Operator,NaN,NaN,Thank you. Our next question comes from Jim Mi...,4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['thank', 'you', 'next', 'question', 'comes', ...",['thank you next question comes jim mitchell s...
4,Jim Mitchell,Q,"Analyst, Seaport Global Securities LLC","Hey. Good morning. Maybe just on regulation, w...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['hey', 'good', 'morning', 'maybe', 'regulatio...",['hey good morning maybe regulation new admini...


In [368]:
#preprocessing data
preprocessor(qa_data, "utterance", "question_tokenised_data", "question_cleaned")
preprocessor(qa_data,"utterance","answer_tokenised_data","answer_cleaned")

In [369]:
#remove operater
#qa_data = qa_data.loc[qa_data["speaker"]!="Operator"]

#remove less than 20 words
qa_data["count"] = qa_data["question_tokenised_data"].apply(lambda x: len(x))
qa_data = qa_data.loc[qa_data["count"]>20]
qa_data.head()

,speaker,marker,job_title,utterance,filename,financial_quarter,call_date,tokenised_data,cleaned_data,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count
0,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","Yeah. I think the conventional wisdom on QT, a...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['yeah', 'think', 'conventional', 'wisdom', 'p...",['yeah think conventional wisdom qt im pretend...,"[yeah, think, conventional, wisdom, pretending...",[yeah think conventional wisdom qt im pretendi...,"[yeah, think, conventional, wisdom, pretending...",[yeah think conventional wisdom qt im pretendi...,80
4,Jim Mitchell,Q,"Analyst, Seaport Global Securities LLC","Hey. Good morning. Maybe just on regulation, w...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['hey', 'good', 'morning', 'maybe', 'regulatio...",['hey good morning maybe regulation new admini...,"[hey, good, morning, maybe, regulation, new, a...",[hey good morning maybe regulation new adminis...,"[hey, good, morning, maybe, regulation, new, a...",[hey good morning maybe regulation new adminis...,39
5,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","Hey, Jim. I mean, it's obviously something we'...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['hey', 'jim', 'mean', 'obviously', 'something...",['hey jim mean obviously something were thinki...,"[hey, jim, mean, obviously, something, re, thi...",[hey jim mean obviously something were thinkin...,"[hey, jim, mean, obviously, something, re, thi...",[hey jim mean obviously something were thinkin...,60
6,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","everything, more capital, more liquidity, that...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['everything', 'capital', 'liquidity', 'uses',...",['everything capital liquidity uses data balan...,"[everything, capital, liquidity, us, data, bal...",[everything capital liquidity uses data balanc...,"[everything, capital, liquidity, us, data, bal...",[everything capital liquidity uses data balanc...,74
7,Jamie Dimon,A,"Chairman & Chief Executive Officer, JPMorganChase","Can I just add, no that's great. Jeremy gave i...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['add', 'that', 'great', 'jeremy', 'gave', 'al...",['add thats great jeremy gave all let add thre...,"[add, that, great, jeremy, gave, all, let, add...",[add thats great jeremy gave all let add three...,"[add, that, great, jeremy, gave, all, let, add...",[add thats great jeremy gave all let add three...,80


In [370]:
#reset index
qa_data.reset_index(drop=True, inplace=True)
qa_data.head()

,speaker,marker,job_title,utterance,filename,financial_quarter,call_date,tokenised_data,cleaned_data,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count
0,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","Yeah. I think the conventional wisdom on QT, a...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['yeah', 'think', 'conventional', 'wisdom', 'p...",['yeah think conventional wisdom qt im pretend...,"[yeah, think, conventional, wisdom, pretending...",[yeah think conventional wisdom qt im pretendi...,"[yeah, think, conventional, wisdom, pretending...",[yeah think conventional wisdom qt im pretendi...,80
1,Jim Mitchell,Q,"Analyst, Seaport Global Securities LLC","Hey. Good morning. Maybe just on regulation, w...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['hey', 'good', 'morning', 'maybe', 'regulatio...",['hey good morning maybe regulation new admini...,"[hey, good, morning, maybe, regulation, new, a...",[hey good morning maybe regulation new adminis...,"[hey, good, morning, maybe, regulation, new, a...",[hey good morning maybe regulation new adminis...,39
2,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","Hey, Jim. I mean, it's obviously something we'...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['hey', 'jim', 'mean', 'obviously', 'something...",['hey jim mean obviously something were thinki...,"[hey, jim, mean, obviously, something, re, thi...",[hey jim mean obviously something were thinkin...,"[hey, jim, mean, obviously, something, re, thi...",[hey jim mean obviously something were thinkin...,60
3,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","everything, more capital, more liquidity, that...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['everything', 'capital', 'liquidity', 'uses',...",['everything capital liquidity uses data balan...,"[everything, capital, liquidity, us, data, bal...",[everything capital liquidity uses data balanc...,"[everything, capital, liquidity, us, data, bal...",[everything capital liquidity uses data balanc...,74
4,Jamie Dimon,A,"Chairman & Chief Executive Officer, JPMorganChase","Can I just add, no that's great. Jeremy gave i...",4q24-earnings-transcript.pdf,4Q24,2025-01-15,"['add', 'that', 'great', 'jeremy', 'gave', 'al...",['add thats great jeremy gave all let add thre...,"[add, that, great, jeremy, gave, all, let, add...",[add thats great jeremy gave all let add three...,"[add, that, great, jeremy, gave, all, let, add...",[add thats great jeremy gave all let add three...,80


In [371]:
#Create question number column
create_ques_num_column(qa_data,"question_number_inline","marker")

In [372]:
#adding new columns
qa_data["analyst"]=None
qa_data["analyst_title"]=None
qa_data["metadata_question"]=None
qa_data["question_num"]=None

for i in qa_data.index:
  #If the value in the "marker" column at row i is "Q", then set the values of "analyst", "analyst_title", "metadata_question", and "question_num" columns at row i to "x".
  if qa_data.loc[i,"marker"]=="Q":
    qa_data.at[i,"analyst"]="x"
    qa_data.at[i,"analyst_title"]="x"
    qa_data.at[i,"metadata_question"]="x"
    qa_data.at[i,"question_num"]="x"
  #Initialize lists and populate them with "speaker", "job_title", "utterance", and "question_number_inline" values from rows between last_a and i in qa_data.
  else:
    last_a = find_row(qa_data,"marker", i)+ 1
    name_list=[]
    title_list=[]
    question_list=[]
    quest_num_list=[]
    for j in range(last_a, i):
      name_list.append(qa_data["speaker"][j])
      title_list.append(qa_data["job_title"][j])
      question_list.append(qa_data["utterance"][j])
      quest_num_list.append(str(qa_data["question_number_inline"][j]))
 # Assign the lists name_list, title_list, question_list, and quest_num_list to the "analyst", "analyst_title", "metadata_question", and "question_num" columns at row i in qa_data, respectively.
    qa_data.at[i,"analyst"]=name_list
    qa_data.at[i,"analyst_title"]=title_list
    qa_data.at[i,"metadata_question"]=question_list
    qa_data.at[i,"question_num"]=quest_num_list

In [373]:
for i in range(len(qa_data)):
  #if the first row is answer, then return all analyst columns as x, if the row is answer and there is not analyst, then popurate questioner analyst name and other data, otherwise pass the function
  if i ==0 and qa_data["marker"][i]=="A":
    qa_data.at[i,"analyst"] ="x"
    qa_data.at[i,"analyst_title"] ="x"
    qa_data.at[i,"metadata_question"] ="x"
    qa_data.at[i,"question_num"]="x"
  elif qa_data["marker"][i]=="A"and qa_data["analyst"][i]==[]:
    a = find_row_empty(qa_data,"marker","analyst",i)
    qa_data.at[i,"analyst"] = qa_data.loc[a,"analyst"]
    qa_data.at[i,"analyst_title"] = qa_data.loc[a,"analyst_title"]
    qa_data.at[i,"metadata_question"] = qa_data.loc[a,"metadata_question"]
    qa_data.at[i,"question_num"] =qa_data.loc[a,"question_num"]
  else:
    continue


In [374]:
#rename column
qa_data.rename(columns={"utterance":"metadata"},inplace=True)


In [375]:
#reorganise column
qa_data=qa_data[["filename","financial_quarter","call_date","speaker","marker","question_num","job_title","metadata", "answer_cleaned","analyst","analyst_title","metadata_question","question_cleaned"]]

In [376]:
qa_data.head(20)

,filename,financial_quarter,call_date,speaker,marker,question_num,job_title,metadata,answer_cleaned,analyst,analyst_title,metadata_question,question_cleaned
0,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jeremy Barnum,A,x,"Chief Financial Officer, JPMorganChase","Yeah. I think the conventional wisdom on QT, a...",[yeah think conventional wisdom qt im pretendi...,x,x,x,[yeah think conventional wisdom qt im pretendi...
1,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jim Mitchell,Q,x,"Analyst, Seaport Global Securities LLC","Hey. Good morning. Maybe just on regulation, w...",[hey good morning maybe regulation new adminis...,x,x,x,[hey good morning maybe regulation new adminis...
2,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jeremy Barnum,A,[0],"Chief Financial Officer, JPMorganChase","Hey, Jim. I mean, it's obviously something we'...",[hey jim mean obviously something were thinkin...,[Jim Mitchell],"[Analyst, Seaport Global Securities LLC]","[Hey. Good morning. Maybe just on regulation, ...",[hey jim mean obviously something were thinkin...
3,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jeremy Barnum,A,[0],"Chief Financial Officer, JPMorganChase","everything, more capital, more liquidity, that...",[everything capital liquidity uses data balanc...,[Jim Mitchell],"[Analyst, Seaport Global Securities LLC]","[Hey. Good morning. Maybe just on regulation, ...",[everything capital liquidity uses data balanc...
4,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jamie Dimon,A,[0],"Chairman & Chief Executive Officer, JPMorganChase","Can I just add, no that's great. Jeremy gave i...",[add thats great jeremy gave all let add three...,[Jim Mitchell],"[Analyst, Seaport Global Securities LLC]","[Hey. Good morning. Maybe just on regulation, ...",[add thats great jeremy gave all let add three...
5,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jim Mitchell,Q,x,"Analyst, Seaport Global Securities LLC","Yeah. No, that makes sense. And maybe just as ...",[yeah no makes sense maybe follow-up loan grow...,x,x,x,[yeah no makes sense maybe follow-up loan grow...
6,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jeremy Barnum,A,[1],"Chief Financial Officer, JPMorganChase","Yeah, it's a good question. And I think given ...",[yeah good question think given significant im...,[Jim Mitchell],"[Analyst, Seaport Global Securities LLC]","[Yeah. No, that makes sense. And maybe just as...",[yeah good question think given significant im...
7,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Erika Najarian,Q,x,"Analyst, UBS Securities LLC","Yes. Hi, good morning. Wanted to follow up on ...",[yes hi good morning wanted follow questions c...,x,x,x,[yes hi good morning wanted follow questions c...
8,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jeremy Barnum,A,[2],"Chief Financial Officer, JPMorganChase","Right, Erika. Okay. You are tempting me with m...",[right erika okay tempting many rabbit holes d...,[Erika Najarian],"[Analyst, UBS Securities LLC]","[Yes. Hi, good morning. Wanted to follow up on...",[right erika okay tempting many rabbit holes d...
9,4q24-earnings-transcript.pdf,4Q24,2025-01-15,Jamie Dimon,A,[2],"Chairman & Chief Executive Officer, JPMorganChase","Oh, yeah. This is an unfortunate thing for any...",[oh yeah unfortunate thing big company like pe...,[Erika Najarian],"[Analyst, UBS Securities LLC]","[Yes. Hi, good morning. Wanted to follow up on...",[oh yeah unfortunate thing big company like pe...


JP morgan management discussion

In [ ]:
%ls

In [ ]:
#defining santader dataframe
jpmorgan_body_df=pd.read_csv("jpmorgan_management_discussion.csv")
jpmorgan_body_df.head()

In [ ]:
#preprocess data
preprocessor(jpmorgan_body_df, "chunk_text", "tokenized_data","cleaned_data")

In [ ]:
jpmorgan_body_df.head()

UBS qna section

In [ ]:
%ls

In [ ]:
#define ubs q&a data
ubs_qna_df=pd.read_csv("ubs_qna_section.csv")

In [ ]:
#preprocessing ubs Q&A data
preprocessor(ubs_qna_df, "utterance", "tokenized_data","cleaned_data")

In [ ]:
ubs_qna_df.head()

UBS management discussion

In [ ]:
%ls

In [ ]:
#defining ubs management discussion
ubs_manag_df=pd.read_csv("ubs_management_discussion.csv")
ubs_manag_df.head()

In [ ]:
#preprocessing ubs management discussion
preprocessor(ubs_manag_df,"utterance", "tokenized_data","cleaned_data")
ubs_manag_df.head()

# **Export the output as a csv file**

JP morgan QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path1 = "/content/bank_of_england/data/preprocessed_data/jpmorgan_qna_df_preprocessed_ver6.csv"
qa_data.to_csv(preprocessed_qa_csv_path1, index=False)

JP morgan management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path2 = "/content/sample_data/jpmorgan_management_df_preprocessed.csv"
jpmorgan_body_df.to_csv(preprocessed_qa_csv_path2, index=False)

UBS QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path3 = "/content/sample_data/ubs_qa_df_preprocessed.csv"
ubs_qna_df.to_csv(preprocessed_qa_csv_path3, index=False)

UBS management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
ubs_manag_df.to_csv(preprocessed_qa_csv_path4, index=False)